In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import os
import datetime

chrome_path = "selenium_driver_chrome/chromedriver.exe" 
username = "Enter yours"
password = "Enter yours"
gmailsername = 'Enter yours'
gmailpassword = 'Enter yours'


web = webdriver.Chrome(chrome_path)

web.get('https://www.fundrich.com.tw/FundWeb/Overview/Investment.aspx')
web.set_window_position(0,0) #瀏覽器位置
web.set_window_size(700,700) #瀏覽器大小
time.sleep(5)

web.find_element_by_xpath("//input[@type='text']").send_keys(username)
web.find_element_by_xpath("//input[@type='password']").send_keys(password)
web.find_element_by_xpath('//*[@id="content"]/div/div/div[2]/div/div[2]/div/div/button').click()
time.sleep(5)
web.get("https://www.fundrich.com.tw/FundWeb/Overview/Investment.aspx")


In [2]:
web.find_element_by_xpath('//*[@id="table1_length"]/label/select/option[3]').click()
time.sleep(2)

spn_INCOME_RATE = web.find_element_by_xpath('//*[@id="spn_INCOME_RATE"]').text

print(spn_INCOME_RATE)

-2.59%


In [3]:
content_data = []
header_data = []
table = web.find_element_by_xpath('//*[@id="table1"]')
table_header = [header.text for header in table.find_elements_by_tag_name('th')]
table_content = [header.text for header in table.find_elements_by_tag_name('td')]


In [4]:
new_table_header=[]
for txt in table_header:
    if txt == '平均\n成本\n參考\n匯率':
        txt = '平均成本\n參考匯率'
    if txt == '交易\n幣別':
        txt = '交易幣別'
    if txt == '報酬率\n(含息)\n報酬率\n(不含息)':
        txt = '報酬率(含息)\n報酬率(不含息)'
    new_table_header.extend(txt.split("\n")) 


In [5]:
new_table_content=[]
for txt in table_content:
    if txt == '買回\n轉換\n轉申購':
        txt =  '買回轉換轉申購'
    new_table_content.extend(txt.split("\n")) 
    
    
new_table_content = np.array(new_table_content).reshape(-1,len(new_table_header))
    

In [6]:
new_df = pd.DataFrame(columns=new_table_header, data=new_table_content)
new_df['報酬率(含息)'] = new_df['報酬率(含息)'].str.replace(r'%', r'').astype('float') 
ReturnWithInterest = np.average(new_df['報酬率(含息)'], weights=new_df['台幣本金'].str.replace(r',', r'').astype('float')/new_df['台幣本金'].str.replace(r',', r'').astype('float').sum())
ReturnWithInterest = "{0:.2f}%".format(ReturnWithInterest)
new_df = new_df.sort_values(by=['報酬率(含息)'])
new_df['報酬率(含息)'] = pd.Series(["{0:.2f}%".format(val ) for val in new_df['報酬率(含息)']], index = new_df.index)

In [7]:
filename = "fundrich-overview/fundrich{0}.csv"
oldfilename = filename.format('')
while (os.path.isfile(oldfilename)):
    v= datetime.datetime.fromtimestamp(os.path.getmtime(oldfilename))
    x = v.strftime('%Y%m%d-%H%M%S')
    try:
        os.rename(oldfilename,filename.format(x))
    except FileExistsError:
        os.remove(filename.format(x))
        os.rename(oldfilename,filename.format(x))

new_df.to_csv(oldfilename, encoding="big5")

In [8]:
filename = "fundrich-overview/fundrichoverview.csv"
v= datetime.datetime.fromtimestamp(os.path.getmtime(oldfilename))
if(not os.path.isfile(filename)):
    f = open(filename,"w")
    f.write("Datatime,Return\n")
    f.close()

f = open(filename,"a")
f.write("{0},{1}\n".format(str(v),spn_INCOME_RATE))
f.close()



In [9]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys



recipients = ['a4455669@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = "Fundrich Summary"
msg['From'] = 'a4455669@gmail.com'


html = """\
<html>
  <head>總投報率(含息):{0} <br/> 總投報率(不含息):{1}   </head>
  <body>
    {2}
  </body>
</html>
""".format(ReturnWithInterest, spn_INCOME_RATE ,new_df.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)


server = smtplib.SMTP('smtp.gmail.com:587')
server.ehlo()
server.starttls()
server.login(gmailsername,gmailpassword)
server.sendmail(msg['From'], emaillist , msg.as_string())

{}

In [10]:
web.quit()